In [ ]:
%%capture
pip install NeuralForecast

In [ ]:

import tensorflow as tf
physical_devices = tf.config.list_physical_devices('GPU')


In [ ]:
physical_devices

[]

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from neuralforecast.models import NHITS
from neuralforecast import NeuralForecast
from neuralforecast.core import NeuralForecast
from neuralforecast.models import Informer, Autoformer,NHITS
from neuralforecast.models import NBEATS,FEDformer

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
X_df = pd.read_csv('/content/drive/MyDrive/AML/ECL/S/df_x.csv')
Y_df = pd.read_csv('/content/drive/MyDrive/AML/ECL/S/df_y.csv')

In [ ]:
X_df

In [ ]:
Y_df

In [ ]:
Y_df['unique_id'].unique()

In [ ]:
Y_df['ex_1']=X_df['ex_1']
Y_df['ex_2']=X_df['ex_2']
Y_df['ex_3']=X_df['ex_3']
Y_df['ex_4']=X_df['ex_4']

Y_df=Y_df.dropna()

In [ ]:
Y_df

In [ ]:
Y_df['ds'] = pd.to_datetime(Y_df['ds'])


## TEMP

In [ ]:
# horizon = 192
# models = [
#           Informer(h=horizon,
#                 input_size=horizon,
#                 max_steps=1000,
#                 val_check_steps=100,
#                 early_stop_patience_steps=3),
#           Autoformer(h=horizon,
#                 input_size=horizon,
#                 max_steps=1000,
#                 val_check_steps=100,
#                 early_stop_patience_steps=3),
#           FEDformer(h=horizon,
#                 input_size=horizon,
#                 max_steps=1000,
#                 val_check_steps=100,
#                 early_stop_patience_steps=3),
#           NBEATS(h=horizon,
#                 input_size=horizon,
#                 max_steps=1000,
#                 val_check_steps=100,
#                 early_stop_patience_steps=3),
#             NHITS(h = horizon,
#                 input_size = horizon,
#                 # stat_exog_list=[''],
#                 # hist_exog_list = ['system_load'],
#                 futr_exog_list=['ex_1','ex_2','ex_3','ex_4'],
#                 max_steps=1000,
#                 val_check_steps=100,
#                 early_stop_patience_steps=3,
#                 scaler_type = 'robust')
#             ]


# TEMP

### Nhits

In [ ]:
from joblib import Parallel, delayed
import time

# Define function for cross-validation
def cross_validate_model(model, Y_df, val_size, test_size, n_windows=None):
    start = time.time()
    nf = NeuralForecast(models=[model], freq='60min')
    Y_hat_df = nf.cross_validation(df=Y_df, val_size=val_size, test_size=test_size, n_windows=n_windows)
    end = time.time()
    tim = (end-start)/60
    # print(f"Cross-validation time for {(end-start)/60:.2f} minutes")
    return Y_hat_df, tim

# Define models
horizon = 96
models = [
            NHITS(h = horizon,
                input_size = horizon,
                # stat_exog_list=[''],
                # hist_exog_list = ['system_load'],
                futr_exog_list=['ex_1','ex_2','ex_3','ex_4'],
                max_steps=1000,
                val_check_steps=100,
                early_stop_patience_steps=3,
                scaler_type = 'robust')
            ]

# Parallelize cross-validation for each model
Results = Parallel(n_jobs=-1)(delayed(cross_validate_model)(model, Y_df, val_size, test_size) for model in models)



In [ ]:
print(f"Cross-validation time for {Results[0][1]} minutes")

In [ ]:
Results

In [ ]:
Y_hat_df = Results[0][0]
Y_hat_df

In [ ]:
import matplotlib.pyplot as plt
Y_plot = Y_hat_df[Y_hat_df['unique_id']=='OT'] # OT dataset
cutoffs = Y_hat_df['cutoff'].unique()[::horizon]
Y_plot = Y_plot[Y_hat_df['cutoff'].isin(cutoffs)]

plt.figure(figsize=(20,5))
plt.plot(Y_plot['ds'], Y_plot['y'], label='True')


# plt.plot(Y_plot['ds'], Y_plot['NBEATS'], label='NBEATS')

# plt.plot(Y_plot['ds'], Y_plot['FEDformer'], label='FEDformer')

plt.plot(Y_plot['ds'], Y_plot['NHITS'], label='NHITS')

# plt.plot(Y_plot['ds'], Y_plot['Informer'], label='Informer')

# plt.plot(Y_plot['ds'], Y_plot['Autoformer'], label='Autoformer')


plt.xlabel('Datestamp')
plt.ylabel('OT')
plt.grid()
plt.legend()

In [ ]:
from neuralforecast.losses.numpy import mae,mse

# mae_autoformer = mae(Y_hat_df['y'], Y_hat_df['Autoformer'])
# mse_autoformer = mse(Y_hat_df['y'], Y_hat_df['Autoformer'])
# print(f'Autoformer MAE: {mae_autoformer:.3f}')
# print(f'Autoformer MSE: {mse_autoformer:.3f}')


# mae_fedformer = mae(Y_hat_df['y'], Y_hat_df['FEDformer'])
# mse_fedformer = mse(Y_hat_df['y'], Y_hat_df['FEDformer'])
# print(f'FEDformer MAE: {mae_fedformer:.3f}')
# print(f'FEDformer MSE: {mse_fedformer:.3f}')

# mae_nbeats = mae(Y_hat_df['y'], Y_hat_df['NBEATS'])
# mse_nbeats = mse(Y_hat_df['y'], Y_hat_df['NBEATS'])
# print(f'NBEATS MAE: {mae_nbeats:.3f}')
# print(f'NBEATS MSE: {mse_nbeats:.3f}')

mae_nhits = mae(Y_hat_df['y'], Y_hat_df['NHITS'])
mse_nhits = mse(Y_hat_df['y'], Y_hat_df['NHITS'])
print(f'NHITS MAE: {mae_nhits:.3f}')
print(f'NHITS MSE: {mse_nhits:.3f}')

# mae_informer = mae(Y_hat_df['y'], Y_hat_df['Informer'])
# mse_informer = mse(Y_hat_df['y'], Y_hat_df['Informer'])
# print(f'Informer MAE: {mae_informer:.3f}')
# print(f'Informer MSE: {mse_informer:.3f}')



In [ ]:
print(A)

### Informer

In [ ]:
from joblib import Parallel, delayed
import time

# Define function for cross-validation
def cross_validate_model(model, Y_df, val_size, test_size, n_windows=None):
    start = time.time()
    nf = NeuralForecast(models=[model], freq='15min')
    Y_hat_df = nf.cross_validation(df=Y_df, val_size=val_size, test_size=test_size, n_windows=n_windows)
    end = time.time()
    tim = (end-start)/60
    # print(f"Cross-validation time for {(end-start)/60:.2f} minutes")
    return Y_hat_df, tim

# Define models
horizon = 192
models = [
            Informer(h=horizon,
                input_size=horizon,
                max_steps=1000,
                val_check_steps=100,
                early_stop_patience_steps=3),
            ]

# Parallelize cross-validation for each model
Results = Parallel(n_jobs=-1)(delayed(cross_validate_model)(model, Y_df, val_size, test_size) for model in models)



In [ ]:
print(f"Cross-validation time for {Results[0][1]} minutes")

In [ ]:
Y_hat_df = Results[0][0]

In [ ]:
import matplotlib.pyplot as plt
Y_plot = Y_hat_df[Y_hat_df['unique_id']=='OT'] # OT dataset
cutoffs = Y_hat_df['cutoff'].unique()[::horizon]
Y_plot = Y_plot[Y_hat_df['cutoff'].isin(cutoffs)]

plt.figure(figsize=(20,5))
plt.plot(Y_plot['ds'], Y_plot['y'], label='True')


# plt.plot(Y_plot['ds'], Y_plot['NBEATS'], label='NBEATS')

# plt.plot(Y_plot['ds'], Y_plot['FEDformer'], label='FEDformer')

# plt.plot(Y_plot['ds'], Y_plot['NHITS'], label='NHITS')

plt.plot(Y_plot['ds'], Y_plot['Informer'], label='Informer')

# plt.plot(Y_plot['ds'], Y_plot['Autoformer'], label='Autoformer')


plt.xlabel('Datestamp')
plt.ylabel('OT')
plt.grid()
plt.legend()

In [ ]:
from neuralforecast.losses.numpy import mae,mse

# mae_autoformer = mae(Y_hat_df['y'], Y_hat_df['Autoformer'])
# mse_autoformer = mse(Y_hat_df['y'], Y_hat_df['Autoformer'])
# print(f'Autoformer MAE: {mae_autoformer:.3f}')
# print(f'Autoformer MSE: {mse_autoformer:.3f}')


# mae_fedformer = mae(Y_hat_df['y'], Y_hat_df['FEDformer'])
# mse_fedformer = mse(Y_hat_df['y'], Y_hat_df['FEDformer'])
# print(f'FEDformer MAE: {mae_fedformer:.3f}')
# print(f'FEDformer MSE: {mse_fedformer:.3f}')

# mae_nbeats = mae(Y_hat_df['y'], Y_hat_df['NBEATS'])
# mse_nbeats = mse(Y_hat_df['y'], Y_hat_df['NBEATS'])
# print(f'NBEATS MAE: {mae_nbeats:.3f}')
# print(f'NBEATS MSE: {mse_nbeats:.3f}')

# mae_nhits = mae(Y_hat_df['y'], Y_hat_df['NHITS'])
# mse_nhits = mse(Y_hat_df['y'], Y_hat_df['NHITS'])
# print(f'NHITS MAE: {mae_nhits:.3f}')
# print(f'NHITS MSE: {mse_nhits:.3f}')

mae_informer = mae(Y_hat_df['y'], Y_hat_df['Informer'])
mse_informer = mse(Y_hat_df['y'], Y_hat_df['Informer'])
print(f'Informer MAE: {mae_informer:.3f}')
print(f'Informer MSE: {mse_informer:.3f}')



### Autoformer

In [ ]:
from joblib import Parallel, delayed
import time

# Define function for cross-validation
def cross_validate_model(model, Y_df, val_size, test_size, n_windows=None):
    start = time.time()
    nf = NeuralForecast(models=[model], freq='15min')
    Y_hat_df = nf.cross_validation(df=Y_df, val_size=val_size, test_size=test_size, n_windows=n_windows)
    end = time.time()
    tim = (end-start)/60
    # print(f"Cross-validation time for {(end-start)/60:.2f} minutes")
    return Y_hat_df, tim

# Define models
horizon = 192
models = [
            Autoformer(h=horizon,
                input_size=horizon,
                max_steps=1000,
                val_check_steps=100,
                early_stop_patience_steps=3),
            ]

# Parallelize cross-validation for each model
Results = Parallel(n_jobs=-1)(delayed(cross_validate_model)(model, Y_df, val_size, test_size) for model in models)



In [ ]:
print(f"Cross-validation time for {Results[0][1]} minutes")

In [ ]:
Y_hat_df = Results[0][0]

In [ ]:
import matplotlib.pyplot as plt
Y_plot = Y_hat_df[Y_hat_df['unique_id']=='OT'] # OT dataset
cutoffs = Y_hat_df['cutoff'].unique()[::horizon]
Y_plot = Y_plot[Y_hat_df['cutoff'].isin(cutoffs)]

plt.figure(figsize=(20,5))
plt.plot(Y_plot['ds'], Y_plot['y'], label='True')


# plt.plot(Y_plot['ds'], Y_plot['NBEATS'], label='NBEATS')

# plt.plot(Y_plot['ds'], Y_plot['FEDformer'], label='FEDformer')

# plt.plot(Y_plot['ds'], Y_plot['NHITS'], label='NHITS')

# plt.plot(Y_plot['ds'], Y_plot['Informer'], label='Informer')

plt.plot(Y_plot['ds'], Y_plot['Autoformer'], label='Autoformer')


plt.xlabel('Datestamp')
plt.ylabel('OT')
plt.grid()
plt.legend()

In [ ]:
from neuralforecast.losses.numpy import mae,mse

mae_autoformer = mae(Y_hat_df['y'], Y_hat_df['Autoformer'])
mse_autoformer = mse(Y_hat_df['y'], Y_hat_df['Autoformer'])
print(f'Autoformer MAE: {mae_autoformer:.3f}')
print(f'Autoformer MSE: {mse_autoformer:.3f}')


# mae_fedformer = mae(Y_hat_df['y'], Y_hat_df['FEDformer'])
# mse_fedformer = mse(Y_hat_df['y'], Y_hat_df['FEDformer'])
# print(f'FEDformer MAE: {mae_fedformer:.3f}')
# print(f'FEDformer MSE: {mse_fedformer:.3f}')

# mae_nbeats = mae(Y_hat_df['y'], Y_hat_df['NBEATS'])
# mse_nbeats = mse(Y_hat_df['y'], Y_hat_df['NBEATS'])
# print(f'NBEATS MAE: {mae_nbeats:.3f}')
# print(f'NBEATS MSE: {mse_nbeats:.3f}')

# mae_nhits = mae(Y_hat_df['y'], Y_hat_df['NHITS'])
# mse_nhits = mse(Y_hat_df['y'], Y_hat_df['NHITS'])
# print(f'NHITS MAE: {mae_nhits:.3f}')
# print(f'NHITS MSE: {mse_nhits:.3f}')

# mae_informer = mae(Y_hat_df['y'], Y_hat_df['Informer'])
# mse_informer = mse(Y_hat_df['y'], Y_hat_df['Informer'])
# print(f'Informer MAE: {mae_informer:.3f}')
# print(f'Informer MSE: {mse_informer:.3f}')



### FEDformer

In [ ]:
from joblib import Parallel, delayed
import time

# Define function for cross-validation
def cross_validate_model(model, Y_df, val_size, test_size, n_windows=None):
    start = time.time()
    nf = NeuralForecast(models=[model], freq='15min')
    Y_hat_df = nf.cross_validation(df=Y_df, val_size=val_size, test_size=test_size, n_windows=n_windows)
    end = time.time()
    tim = (end-start)/60
    # print(f"Cross-validation time for {(end-start)/60:.2f} minutes")
    return Y_hat_df, tim

# Define models
horizon = 192
models = [
             FEDformer(h=horizon,
                input_size=horizon,
                max_steps=1000,
                val_check_steps=100,
                early_stop_patience_steps=3),
            ]

# Parallelize cross-validation for each model
Results = Parallel(n_jobs=-1)(delayed(cross_validate_model)(model, Y_df, val_size, test_size) for model in models)



In [ ]:
print(f"Cross-validation time for {Results[0][1]} minutes")

In [ ]:
Y_hat_df = Results[0][0]

In [ ]:
import matplotlib.pyplot as plt
Y_plot = Y_hat_df[Y_hat_df['unique_id']=='OT'] # OT dataset
cutoffs = Y_hat_df['cutoff'].unique()[::horizon]
Y_plot = Y_plot[Y_hat_df['cutoff'].isin(cutoffs)]

plt.figure(figsize=(20,5))
plt.plot(Y_plot['ds'], Y_plot['y'], label='True')


# plt.plot(Y_plot['ds'], Y_plot['NBEATS'], label='NBEATS')

plt.plot(Y_plot['ds'], Y_plot['FEDformer'], label='FEDformer')

# plt.plot(Y_plot['ds'], Y_plot['NHITS'], label='NHITS')

# plt.plot(Y_plot['ds'], Y_plot['Informer'], label='Informer')

# plt.plot(Y_plot['ds'], Y_plot['Autoformer'], label='Autoformer')


plt.xlabel('Datestamp')
plt.ylabel('OT')
plt.grid()
plt.legend()

In [ ]:
from neuralforecast.losses.numpy import mae,mse

# mae_autoformer = mae(Y_hat_df['y'], Y_hat_df['Autoformer'])
# mse_autoformer = mse(Y_hat_df['y'], Y_hat_df['Autoformer'])
# print(f'Autoformer MAE: {mae_autoformer:.3f}')
# print(f'Autoformer MSE: {mse_autoformer:.3f}')


mae_fedformer = mae(Y_hat_df['y'], Y_hat_df['FEDformer'])
mse_fedformer = mse(Y_hat_df['y'], Y_hat_df['FEDformer'])
print(f'FEDformer MAE: {mae_fedformer:.3f}')
print(f'FEDformer MSE: {mse_fedformer:.3f}')

# mae_nbeats = mae(Y_hat_df['y'], Y_hat_df['NBEATS'])
# mse_nbeats = mse(Y_hat_df['y'], Y_hat_df['NBEATS'])
# print(f'NBEATS MAE: {mae_nbeats:.3f}')
# print(f'NBEATS MSE: {mse_nbeats:.3f}')

# mae_nhits = mae(Y_hat_df['y'], Y_hat_df['NHITS'])
# mse_nhits = mse(Y_hat_df['y'], Y_hat_df['NHITS'])
# print(f'NHITS MAE: {mae_nhits:.3f}')
# print(f'NHITS MSE: {mse_nhits:.3f}')

# mae_informer = mae(Y_hat_df['y'], Y_hat_df['Informer'])
# mse_informer = mse(Y_hat_df['y'], Y_hat_df['Informer'])
# print(f'Informer MAE: {mae_informer:.3f}')
# print(f'Informer MSE: {mse_informer:.3f}')



### NBEATS

In [ ]:
from joblib import Parallel, delayed
import time

# Define function for cross-validation
def cross_validate_model(model, Y_df, val_size, test_size, n_windows=None):
    start = time.time()
    nf = NeuralForecast(models=[model], freq='15min')
    Y_hat_df = nf.cross_validation(df=Y_df, val_size=val_size, test_size=test_size, n_windows=n_windows)
    end = time.time()
    tim = (end-start)/60
    # print(f"Cross-validation time for {(end-start)/60:.2f} minutes")
    return Y_hat_df, tim

# Define models
horizon = 192
models = [
            NBEATS(h=horizon,
                input_size=horizon,
                max_steps=1000,
                val_check_steps=100,
                early_stop_patience_steps=3),
            ]

# Parallelize cross-validation for each model
Results = Parallel(n_jobs=-1)(delayed(cross_validate_model)(model, Y_df, val_size, test_size) for model in models)



In [ ]:
print(f"Cross-validation time for {Results[0][1]} minutes")

In [ ]:
Y_hat_df = Results[0][0]

In [ ]:
import matplotlib.pyplot as plt
Y_plot = Y_hat_df[Y_hat_df['unique_id']=='OT'] # OT dataset
cutoffs = Y_hat_df['cutoff'].unique()[::horizon]
Y_plot = Y_plot[Y_hat_df['cutoff'].isin(cutoffs)]

plt.figure(figsize=(20,5))
plt.plot(Y_plot['ds'], Y_plot['y'], label='True')


plt.plot(Y_plot['ds'], Y_plot['NBEATS'], label='NBEATS')

# plt.plot(Y_plot['ds'], Y_plot['FEDformer'], label='FEDformer')

# plt.plot(Y_plot['ds'], Y_plot['NHITS'], label='NHITS')

# plt.plot(Y_plot['ds'], Y_plot['Informer'], label='Informer')

# plt.plot(Y_plot['ds'], Y_plot['Autoformer'], label='Autoformer')


plt.xlabel('Datestamp')
plt.ylabel('OT')
plt.grid()
plt.legend()

In [ ]:
from neuralforecast.losses.numpy import mae,mse

# mae_autoformer = mae(Y_hat_df['y'], Y_hat_df['Autoformer'])
# mse_autoformer = mse(Y_hat_df['y'], Y_hat_df['Autoformer'])
# print(f'Autoformer MAE: {mae_autoformer:.3f}')
# print(f'Autoformer MSE: {mse_autoformer:.3f}')


# mae_fedformer = mae(Y_hat_df['y'], Y_hat_df['FEDformer'])
# mse_fedformer = mse(Y_hat_df['y'], Y_hat_df['FEDformer'])
# print(f'FEDformer MAE: {mae_fedformer:.3f}')
# print(f'FEDformer MSE: {mse_fedformer:.3f}')

mae_nbeats = mae(Y_hat_df['y'], Y_hat_df['NBEATS'])
mse_nbeats = mse(Y_hat_df['y'], Y_hat_df['NBEATS'])
print(f'NBEATS MAE: {mae_nbeats:.3f}')
print(f'NBEATS MSE: {mse_nbeats:.3f}')

# mae_nhits = mae(Y_hat_df['y'], Y_hat_df['NHITS'])
# mse_nhits = mse(Y_hat_df['y'], Y_hat_df['NHITS'])
# print(f'NHITS MAE: {mae_nhits:.3f}')
# print(f'NHITS MSE: {mse_nhits:.3f}')

# mae_informer = mae(Y_hat_df['y'], Y_hat_df['Informer'])
# mse_informer = mse(Y_hat_df['y'], Y_hat_df['Informer'])
# print(f'Informer MAE: {mae_informer:.3f}')
# print(f'Informer MSE: {mse_informer:.3f}')



# AutoImformr Horizon = 96

In [ ]:
Y_df

In [ ]:
Y_train_df = Y_df.iloc[:26208,:]
Y_test_df = Y_df.iloc[26208:,:]

In [ ]:
Y_test_df = Y_test_df.reset_index()

In [ ]:
Y_test_df.drop(['index'], axis=1, inplace=True)
Y_test_df

In [ ]:
from neuralforecast.losses.numpy import mae,mse
model = Autoformer(h=96,
                 input_size=24,
                #  hidden_size = 16,
                #  conv_hidden_size = 32,
                #  n_head=2,
                 scaler_type='robust',
                 learning_rate=1e-3,
                 max_steps=1000,
                #  val_check_steps=50,
                #  early_stop_patience_steps=2
                   )

In [ ]:
nf = NeuralForecast(
    models=[model],
    freq='H'
)

In [ ]:
nf.fit(df=Y_train_df)
forecasts = nf.predict()


In [ ]:
forecasts['ds'] = [i for i in range(len(forecasts))]
forecasts

In [ ]:
fig, ax = plt.subplots(1, 1, figsize = (15, 7))
plot_df = forecasts.set_index('ds')
Y_test_df[['y']].plot(ax=ax,linewidth=2)
plot_df[[ 'Autoformer']].plot(ax=ax, linewidth=2)

In [ ]:
forecasts=forecasts.set_index('ds')

In [ ]:
from neuralforecast.losses.numpy import mae,mse


mae_autoformer = mae(Y_test_df['y'], forecasts['Autoformer'])
mse_autoformer = mse(Y_test_df['y'], forecasts['Autoformer'])
print(f'Autoformer MAE: {mae_autoformer:.3f}')
print(f'Autoformer MSE: {mse_autoformer:.3f}')

## Horizon = 192

In [ ]:
Y_df

In [ ]:
Y_train_df = Y_df.iloc[:26112,:]
Y_test_df = Y_df.iloc[26112:,:]
Y_test_df

In [ ]:
Y_test_df = Y_test_df.reset_index()

In [ ]:
Y_test_df.drop(['index'], axis=1, inplace=True)
Y_test_df

In [ ]:
from neuralforecast.losses.numpy import mae,mse
model = Autoformer(h=192,
                 input_size=24,
                #  hidden_size = 16,
                #  conv_hidden_size = 32,
                #  n_head=2,
                 scaler_type='robust',
                 learning_rate=1e-3,
                 max_steps=1,
                #  val_check_steps=50,
                #  early_stop_patience_steps=2
                   )

In [ ]:
nf = NeuralForecast(
    models=[model],
    freq='H'
)

In [ ]:
nf.fit(df=Y_train_df)
forecasts = nf.predict()


In [ ]:
forecasts['ds'] = [i for i in range(len(forecasts))]
forecasts

In [ ]:
fig, ax = plt.subplots(1, 1, figsize = (15, 7))
plot_df = forecasts.set_index('ds')
Y_test_df[['y']].plot(ax=ax,linewidth=2)
plot_df[[ 'Autoformer']].plot(ax=ax, linewidth=2)

In [ ]:
forecasts=forecasts.set_index('ds')

In [ ]:
from neuralforecast.losses.numpy import mae,mse


mae_autoformer = mae(Y_test_df['y'], forecasts['Autoformer'])
mse_autoformer = mse(Y_test_df['y'], forecasts['Autoformer'])
print(f'Autoformer MAE: {mae_autoformer:.3f}')
print(f'Autoformer MSE: {mse_autoformer:.3f}')

## Horizon 336

In [ ]:
Y_df

In [ ]:
Y_train_df = Y_df.iloc[:25968,:]
Y_test_df = Y_df.iloc[25968:,:]
Y_test_df

In [ ]:
Y_test_df = Y_test_df.reset_index()

In [ ]:
Y_test_df.drop(['index'], axis=1, inplace=True)
Y_test_df

In [ ]:
from neuralforecast.losses.numpy import mae,mse
model = Autoformer(h=336,
                 input_size=24,
                #  hidden_size = 16,
                #  conv_hidden_size = 32,
                #  n_head=2,
                 scaler_type='robust',
                 learning_rate=1e-3,
                 max_steps=1,
                #  val_check_steps=50,
                #  early_stop_patience_steps=2
                   )

In [ ]:
nf = NeuralForecast(
    models=[model],
    freq='H'
)

In [ ]:
nf.fit(df=Y_train_df)
forecasts = nf.predict()


In [ ]:
forecasts['ds'] = [i for i in range(len(forecasts))]
forecasts

In [ ]:
fig, ax = plt.subplots(1, 1, figsize = (15, 7))
plot_df = forecasts.set_index('ds')
Y_test_df[['y']].plot(ax=ax,linewidth=2)
plot_df[[ 'Autoformer']].plot(ax=ax, linewidth=2)

In [ ]:
forecasts=forecasts.set_index('ds')

In [ ]:
from neuralforecast.losses.numpy import mae,mse


mae_autoformer = mae(Y_test_df['y'], forecasts['Autoformer'])
mse_autoformer = mse(Y_test_df['y'], forecasts['Autoformer'])
print(f'Autoformer MAE: {mae_autoformer:.3f}')
print(f'Autoformer MSE: {mse_autoformer:.3f}')

## Horizon 720

In [ ]:
Y_df

In [ ]:
Y_train_df = Y_df.iloc[:25584,:]
Y_test_df = Y_df.iloc[25584:,:]
Y_test_df

In [ ]:
Y_test_df = Y_test_df.reset_index()

In [ ]:
Y_test_df.drop(['index'], axis=1, inplace=True)
Y_test_df

In [ ]:
from neuralforecast.losses.numpy import mae,mse
model = Autoformer(h=720,
                 input_size=24,
                #  hidden_size = 16,
                #  conv_hidden_size = 32,
                #  n_head=2,
                 scaler_type='robust',
                 learning_rate=1e-3,
                 max_steps=1,
                #  val_check_steps=50,
                #  early_stop_patience_steps=2
                   )

In [ ]:
nf = NeuralForecast(
    models=[model],
    freq='H'
)

In [ ]:
nf.fit(df=Y_train_df)
forecasts = nf.predict()


In [ ]:
forecasts['ds'] = [i for i in range(len(forecasts))]
forecasts

In [ ]:
fig, ax = plt.subplots(1, 1, figsize = (15, 7))
plot_df = forecasts.set_index('ds')
Y_test_df[['y']].plot(ax=ax,linewidth=2)
plot_df[[ 'Autoformer']].plot(ax=ax, linewidth=2)

In [ ]:
forecasts=forecasts.set_index('ds')

In [ ]:
from neuralforecast.losses.numpy import mae,mse


mae_autoformer = mae(Y_test_df['y'], forecasts['Autoformer'])
mse_autoformer = mse(Y_test_df['y'], forecasts['Autoformer'])
print(f'Autoformer MAE: {mae_autoformer:.3f}')
print(f'Autoformer MSE: {mse_autoformer:.3f}')